# Experiment Notebook: FEI Ecosystem Model Exploratory Analysis

# Table of Contents
* [Experiment Summary](#Experiment-Summary)
* [Experiment Assumptions](#Experiment-Assumptions)
* [Experiment Setup](#Experiment-Setup)
* [Analysis 1: ...](#Analysis-1:-...)

# Experiment Summary 

The purpose of this notebook is to assess the impact that various settings of the ETH price trajectory process have on the main elements of the FEI ecosystem model. 

User FEI Capital Allocation across Liquidity Pool, Money Market, and FEI Savings Deposits.

# Experiment Assumptions

See [assumptions document](../../ASSUMPTIONS.md) for further details.

# Experiment Setup

We begin with several experiment-notebook-level preparatory setup operations:

* Import relevant dependencies
* Import relevant experiment templates
* Create copies of experiments
* Configure and customize experiments 

Analysis-specific setup operations are handled in their respective notebook sections.

In [ ]:
# Import the setup module:
# * sets up the Python path
# * runs shared notebook configuration methods, such as loading IPython modules
import setup

import copy
import logging
import numpy as np
import pandas as pd
import plotly.express as px

import experiments.notebooks.visualizations as visualizations
from experiments.run import run
from experiments.utils import display_code

from experiments.notebooks.helpers.system_metrics import *


In [ ]:
from operator import lt, gt

In [ ]:
from scipy.stats import norm

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
import plotly.io as pio
png_renderer = pio.renderers["png"]
png_renderer.width = 1200
png_renderer.height = 500
# png_renderer.scale = 1

pio.renderers.default = "png"

In [ ]:
# Enable/disable logging
logger = logging.getLogger()
logger.disabled = False

In [ ]:
# Import experiment templates
import experiments.default_experiment as default_experiment

# Analysis 1: Effect of Volatile Asset Trajectory on Main KPIs

The FEI ecosystem model is strongly driven by the Volatile Asset price process - mimicking the dependency on the Ethereum price which avenues of FEI and PCV have in the ecosystem.

In this analysis we look at the effect of setting the volatile asset trajectory in three ways:
- As a linear constant process with stochastic variation 
- As a linear uptrend process with stochastic variation 
- As a linear downtrend process with stochastic variation 

We look at the effect this has on the main ecosystem dynamics in absence of specific monetary policy actions taken by protocol governance, and evaluate downstream effects on mechanism-specific metrics.

In [ ]:
# Create a simulation for each analysis
simulation_1 = copy.deepcopy(default_experiment.experiment.simulations[0])

In [ ]:
cam_deposits = [
    "fei_liquidity_pool_user_deposit",
    "fei_money_market_user_deposit",
    "fei_savings_user_deposit",
    "fei_idle_user_deposit",
]

In [ ]:
parameter_overrides = {
#     "target_rebalancing_condition": [gt, lt], # Simulate decrease and increase of stable PCV
#     "target_stable_pcv_ratio": [0.2, 0.5], # Simulate decrease and increase of stable PCV
#     "rebalancing_period": [int(365 / 4)],
#     "yield_withdrawal_period": [999],  # Disable yield-withdrawal policy
#     "yield_reinvest_period": [999],  # Disable yield-reinvestment policy
    "capital_allocation_fei_deposit_variables": [
            cam_deposits,
    ],
    "capital_allocation_rebalance_duration": [30],
    "fei_savings_rate_process": [
#          lambda _run, timestep: 0.01,
          lambda _run, timestep: 0.03,
#         lambda _run, timestep: 0.01 if timestep < 365 / 4 else (0.03 if timestep < 365 * 3/4 else 0.01),
    ],
    "volatile_asset_price_process": [
        lambda _run, timestep: 2_000 + gen_norm_rv(timestep, 1, 30),
        # lambda _run, timestep: 2_000 if timestep < 365 / 4 else (1_000 if timestep < 365 * 3/4 else 2_000),
        lambda _run, timestep: 2_000 * (1 + timestep * 0.2 / 365) + gen_norm_rv(timestep, 1, 30),
        lambda _run, timestep: 2_000 * (1 - timestep * 0.2 / 365) + gen_norm_rv(timestep, 1, 30),
    ],
    "target_stable_backing_ratio": [0.05], #DEBUG - does not work with 0 or None
    "target_rebalancing_condition": [lt],
    "rebalancing_period": [int(365/4)], # Set to > timesteps to disable policy
    "yield_withdrawal_period": [int(365/4)],  # Toggle manually between policies in state update blocks
    "yield_reinvest_period": [int(365/4)],
    #"money_market_utilization_rate_process": [
    #    lambda _run, timestep: 0.7, #+ gen_norm_rv(timestep, 0, 0.01),
    #]
}


In [ ]:
# Experiment configuration

# Override default experiment number of Monte Carlo Runs
simulation_1.runs = 1

# Override default experiment System Initial State
simulation_1.model.initial_state.update({})

# Override default experiment System Parameters
simulation_1.model.params.update(parameter_overrides)

In [ ]:
# Analysis-specific setup

In [ ]:
# Experiment execution
df, exceptions = run(simulation_1)

In [ ]:
# Post-processing and visualizations

## Visualization of parameter sweep:

We sweep the volatile asset trajectory process for three trends - constant, downtrend, and uptrend, corresponding to stylized market conditions of a sideways, bear, and bull market respectively.

In [ ]:
df.plot(x='timestamp', y=['volatile_asset_price'], color='subset')

# Key KPI analysis

In [ ]:
use_cols = [key + '_balance' for key in cam_deposits]

In [ ]:
fn_dict = get_fn_dict()

### 1. Collateralization Ratio

In [ ]:
# TODO: see what this looks like for < 1 -> reduce total PCV value or initial amounts of FEI
# NICE to have: be able to toggle this from notebook

In [ ]:
fig = get_averages_by_subset(df, ['collateralization_ratio']).plot(
    #x='timestep',
    y='collateralization_ratio',
    color='subset'
)

fig.update_layout(
    title="Collateralization Ratio",
    xaxis_title="Timestamp",
    yaxis_title="Collateralization Ratio",
    legend=dict(
        yanchor="top",
        y=0.98,
        xanchor="left",
        x=0.01
    )
)

fig.show()

In [ ]:
compute_metric_set_for_variable(df, fn_dict, 'collateralization_ratio')

### 2. Stable Backing Ratio

In [ ]:
# NOTE: policy disabled

In [ ]:
fig = get_averages_by_subset(df, ['stable_backing_ratio']).plot(
    #x='timestep',
    y='stable_backing_ratio',
    color='subset'
)

fig.update_layout(
    title="Stable Backing Ratio",
    xaxis_title="Timestamp",
    yaxis_title="Stable Backing Ratio",
    legend=dict(
        yanchor="top",
        y=0.98,
        xanchor="left",
        x=0.01
    )
)

fig.show()

In [ ]:
compute_metric_set_for_variable(df, fn_dict, 'stable_backing_ratio')

### 3. Stable PCV Ratio

In [ ]:
# NOTE: policy disabled

In [ ]:
fig = get_averages_by_subset(df, ['stable_pcv_ratio']).plot(
    #x='timestep',
    y='stable_pcv_ratio',
    color='subset'
)

fig.update_layout(
    title="Stable PCV Ratio",
    xaxis_title="Timestamp",
    yaxis_title="Stable Backing Ratio",
    legend=dict(
        yanchor="top",
        y=0.98,
        xanchor="left",
        x=0.01
    )
)

fig.show()

In [ ]:
compute_metric_set_for_variable(df, fn_dict, 'stable_pcv_ratio')

### 4. Total PCV

In [ ]:
# NOTE: policy disabled

In [ ]:
fig = get_averages_by_subset(df, ['total_pcv']).plot(
    #x='timestep',
    y='total_pcv',
    color='subset'
)

fig.update_layout(
    title="Total PCV",
    xaxis_title="Timestamp",
    yaxis_title="Total PCV",
    legend=dict(
        yanchor="top",
        y=0.98,
        xanchor="left",
        x=0.01
    )
)

fig.show()

In [ ]:
compute_metric_set_for_variable(df, fn_dict, 'total_pcv')

### 5. Total User-Circulating FEI

In [ ]:
fig = get_averages_by_subset(df, ['total_user_circulating_fei']).plot(
    #x='timestep',
    y='total_user_circulating_fei',
    color='subset'
)

fig.update_layout(
    title="Total User Circulating FEI",
    xaxis_title="Timestamp",
    yaxis_title="Total User Circulating FEI",
    legend=dict(
        yanchor="top",
        y=0.98,
        xanchor="left",
        x=0.01
    )
)

fig.show()

In [ ]:
compute_metric_set_for_variable(df, fn_dict, 'total_user_circulating_fei')

### User Circulating FEI Constituents

We look into the constituents of User-Circulating FEI as a result of FEI Capital Allocation Model movements (expanded upon in analysis X (TODO: link))

#### User FEI Liquidity Pool

In [ ]:
fig = get_averages_by_subset(df, ['fei_liquidity_pool_user_deposit_balance']).plot(
    #x='timestamp',
    y='fei_liquidity_pool_user_deposit_balance',
    color='subset',
)

fig.update_layout(
    title='User FEI Liquidity Pool Balance',
    xaxis_title="Timestamp",
    yaxis_title="User FEI LP Balance",

    legend=dict(
        yanchor="top",
        y=0.98,
        xanchor="left",
        x=0.01
    )
)

fig.show()

In [ ]:
compute_metric_set_for_variable(df, fn_dict, 'fei_liquidity_pool_user_deposit_balance')

#### User FEI Money Market

In [ ]:
fig = get_averages_by_subset(df, ['fei_money_market_user_deposit_balance']).plot(
    #x='timestamp',
    y='fei_money_market_user_deposit_balance',
    color='subset',
)

fig.update_layout(
    title='User FEI Money Market Balance',
    xaxis_title="Timestamp",
    yaxis_title="User FEI MM Balance",

    legend=dict(
        yanchor="top",
        y=0.98,
        xanchor="left",
        x=0.01
    )
)

fig.show()

In [ ]:
compute_metric_set_for_variable(df, fn_dict, 'fei_money_market_user_deposit_balance')

#### User FEI Savings Deposit

In [ ]:
fig = get_averages_by_subset(df, ['fei_savings_user_deposit_balance']).plot(
    #x='timestep',
    y='fei_savings_user_deposit_balance',
    color='subset',
)

fig.update_layout(
    title='User FEI Savings Deposit Balance',
    xaxis_title="Timestamp",
    yaxis_title="User FEI SD Balance",

    legend=dict(
        yanchor="top",
        y=0.98,
        xanchor="left",
        x=0.01
    )
)

fig.show()

In [ ]:
compute_metric_set_for_variable(df, fn_dict, 'fei_savings_user_deposit_balance')

#### User Idle FEI

In [ ]:
fig = get_averages_by_subset(df, ['fei_idle_user_deposit_balance']).plot(
    #x='timestep',
    y='fei_idle_user_deposit_balance',
    color='subset',
)

fig.update_layout(
    title='User FEI Savings Idle Balance',
    xaxis_title="Timestamp",
    yaxis_title="User FEI Idle Balance",

    legend=dict(
        yanchor="top",
        y=0.98,
        xanchor="left",
        x=0.01
    )
)

fig.show()

In [ ]:
compute_metric_set_for_variable(df, fn_dict, 'fei_savings_user_deposit_balance')

# Mechanism-Specific KPIs

Here we focus on relevant state variables which encompass the dynamics of the stylized mechanisms in which FEI is deployed

### 1. Money markets

In [ ]:
# TODO: draw conclusion based on inclusion of dependent MM rate

In [ ]:
get_averages_by_subset(df, ['fei_money_market_utilization']).plot(
    #x='timestamp',
    y=['fei_money_market_utilization'],
    color='subset'
)

In [ ]:
compute_metric_set_for_variable(df, fn_dict, 'fei_money_market_utilization')

In [ ]:
get_averages_by_subset(df, ['fei_money_market_borrowed']).plot(
    #x='timestamp',
    y=['fei_money_market_borrowed'],
    color='subset'
)

In [ ]:
compute_metric_set_for_variable(df, fn_dict, 'fei_money_market_borrowed')

In [ ]:
get_averages_by_subset(df, ['fei_money_market_supply_rate', 'fei_money_market_borrow_rate']).plot(
    #x='timestamp',
    y=['fei_money_market_supply_rate', 'fei_money_market_borrow_rate'],
    color='subset'
)

In [ ]:
compute_metric_set_for_variable(df, fn_dict, 'fei_money_market_supply_rate')

In [ ]:
compute_metric_set_for_variable(df, fn_dict, 'fei_money_market_borrow_rate')

### 2. FEI-Volatile Asset Liquidity Pool

In [ ]:
# TODO: explain how this is calculated

In [ ]:
get_averages_by_subset(df, ['fei_liquidity_pool_user_deposit_yield_rate']).plot(
    #x='timestamp',
    y=['fei_liquidity_pool_user_deposit_yield_rate'],
    color='subset'
)

In [ ]:
compute_metric_set_for_variable(df, fn_dict, 'fei_liquidity_pool_user_deposit_yield_rate')

### 3. FEI Savings Rate

In [ ]:
# TODO: explain what it is and how it affects the rest of the system

In [ ]:
df.query('run==1').plot(x='timestamp', y=['fei_savings_rate'], color='subset')

### 4. FEI Issuance/Leverage (minting and redemption)

In [ ]:
# TODO: TBA

### 5. PCV Yield

In [ ]:
# TODO: generalize from 'subset == 0'

In [ ]:
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(y=df.query('subset==0').volatile_yield_bearing_pcv_deposit_yield_accrued, name="Volatile deposit yield-bearing yield accrued"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(y=df.query('subset==0').volatile_idle_pcv_deposit_balance, name="Volatile deposit idle balance"),
    secondary_y=True,
)

In [ ]:
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=df.timestamp, y=df.query('subset==0').volatile_yield_bearing_pcv_deposit_yield_value, name="Volatile deposit yield-bearing yield value"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=df.timestamp, y=df.query('subset==0').volatile_idle_pcv_deposit_asset_value, name="Volatile deposit idle value"),
    secondary_y=True,
)

fig.update_layout(
    title="USD Value of Yield-Bearing Volatile Deposit Balance and Yield Value",
    xaxis_title="Timestamp",
    yaxis_title="USD Value of Deposit Balance and Yield",
    autosize=False,
    width=1000,
    height=675,
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="right",
        x=0.92
    )
)

fig.update_xaxes(title='Timestamp')

## Conclusion

TBA

# Analysis 2: Effect of PCV Management on CR

The FEI ecosystem model is strongly driven by the Volatile Asset price process - mimicking the dependency on the Ethereum price which avenues of FEI and PCV have in the ecosystem.

In this analysis we look at the effect of setting the volatile asset trajectory in three ways:
- As a linear constant process with stochastic variation 
- As a linear uptrend process with stochastic variation 
- As a linear downtrend process with stochastic variation 

We look at the effect this has on the main ecosystem dynamics in absence of specific monetary policy actions taken by protocol governance, and evaluate downstream effects on mechanism-specific metrics.

In [ ]:
# Create a simulation for each analysis
simulation_2 = copy.deepcopy(default_experiment.experiment.simulations[0])

In [ ]:
cam_deposits = [
    "fei_liquidity_pool_user_deposit",
    "fei_money_market_user_deposit",
    "fei_savings_user_deposit",
    "fei_idle_user_deposit",
]

In [ ]:
parameter_overrides = {
#     "target_rebalancing_condition": [gt, lt], # Simulate decrease and increase of stable PCV
#     "target_stable_pcv_ratio": [0.2, 0.5], # Simulate decrease and increase of stable PCV
#     "rebalancing_period": [int(365 / 4)],
#     "yield_withdrawal_period": [999],  # Disable yield-withdrawal policy
#     "yield_reinvest_period": [999],  # Disable yield-reinvestment policy
    "capital_allocation_fei_deposit_variables": [
            cam_deposits,
    ],
    "capital_allocation_rebalance_duration": [30],
    "fei_savings_rate_process": [
#          lambda _run, timestep: 0.01,
          lambda _run, timestep: 0.03,
#         lambda _run, timestep: 0.01 if timestep < 365 / 4 else (0.03 if timestep < 365 * 3/4 else 0.01),
    ],
#     "volatile_asset_price_process": [
#         lambda _run, timestep: 2_000 + gen_norm_rv(timestep, 1, 30),
#         # lambda _run, timestep: 2_000 if timestep < 365 / 4 else (1_000 if timestep < 365 * 3/4 else 2_000),
#         lambda _run, timestep: 2_000 * (1 + timestep * 0.2 / 365) + gen_norm_rv(timestep, 1, 30),
#         lambda _run, timestep: 2_000 * (1 - timestep * 0.2 / 365) + gen_norm_rv(timestep, 1, 30),
#     ],
    "target_rebalancing_condition": [gt, lt], # Simulate decrease and increase of stable PCV
    # Disable policy by setting to None
    "target_stable_pcv_ratio": [None], # Simulate decrease and increase of stable PCV
    "target_stable_backing_ratio": [0.3, 0.8], # Simulate decrease and increase of stable backing
    "rebalancing_period": [int(365/4)], # Set to > timesteps to disable policy
    "yield_withdrawal_period": [int(365/4)],  # Toggle manually between policies in state update blocks
    "yield_reinvest_period": [int(365/4)],
    #"money_market_utilization_rate_process": [
    #    lambda _run, timestep: 0.7, #+ gen_norm_rv(timestep, 0, 0.01),
    #]
}


In [ ]:
# Experiment configuration

# Override default experiment number of Monte Carlo Runs
simulation_2.runs = 100

# Override default experiment System Initial State
simulation_2.model.initial_state.update({})

# Override default experiment System Parameters
simulation_2.model.params.update(parameter_overrides)

In [ ]:
# Analysis-specific setup

In [ ]:
# Experiment execution
df2, exceptions = run(simulation_2)

In [ ]:
def compute_means(df, variable):
    
    
    n_runs = len(df['run'].value_counts())
    
    mu1, mu2 = [], []
    
    for run in range(1,n_runs+1):
        diff = (df.query('subset == 0 and run==@run')[variable] -
                df2.query('subset == 1 and run==@run')[variable])
        
        s1 = df.query('subset == 0 and run==@run')[variable]
        s2 = df.query('subset == 1 and run==@run')[variable]
        
        mu1.append(s1.mean())#/s1.std())
        mu2.append(s2.mean())#/s2.std())
        
    return np.array(mu1), np.array(mu2)

In [ ]:
def compute_sr(df, variable):
    
    
    n_runs = len(df['run'].value_counts())
    
    mu1, mu2 = [], []
    
    for run in range(1,n_runs+1):
        diff = (df.query('subset == 0 and run==@run')[variable] -
                df2.query('subset == 1 and run==@run')[variable])
        
        r1 = df.query('subset == 0 and run==@run')[variable]#.pct_change()
        r2 = df.query('subset == 1 and run==@run')[variable]#.pct_change()
        
        mu1.append(r1.mean()/r1.std())
        mu2.append(r2.mean()/r2.std())
        
    return np.array(mu1), np.array(mu2)

### a

In [ ]:
df2.query('subset == 0').plot(x='timestamp', y=['volatile_asset_price'], color='run')

In [ ]:
fig = get_averages_by_subset(df2, ['collateralization_ratio']).plot(
    #x='timestep',
    y='collateralization_ratio',
    color='subset'
)

fig.update_layout(
    title="Collateralization Ratio",
    xaxis_title="Timestamp",
    yaxis_title="Collateralization Ratio",
    legend=dict(
        yanchor="top",
        y=0.98,
        xanchor="left",
        x=0.01
    )
)

fig.show()

In [ ]:
mu1, mu2 = compute_means(df2, 'collateralization_ratio')

In [ ]:
sr1, sr2 = compute_sr(df2, 'collateralization_ratio')

In [ ]:
a = mu1 >= mu2
prob = a.sum()/len(a)

print('The empirical probability of CR being higher on average with policy 1 than policy 2 is', prob)

In [ ]:
b = sr1 >= sr2
prob = b.sum()/len(b)

print('The empirical probability of CR Sharpe being higher with policy 1 than policy 2 is', prob)

In [ ]:
pd.DataFrame([mu1, mu2]).T.hist(bins=30)

In [ ]:
pd.DataFrame([sr1, sr2]).T.hist(bins=30)

### PCV at Risk

In [ ]:
def calculate_VaR(df, state_variable, alpha, timesteps):
    results = pd.DataFrame()

    for simulation in df.simulation.unique():
        df_simulation = df.query("simulation == @simulation")
        for subset in df_simulation.subset.unique():
            df_subset = df_simulation.query("subset == @subset")
            for run in df_subset.run.unique():
                df_run = df_subset.query("run == @run")

                returns = df_run[state_variable].pct_change()
                final_value = df_run[state_variable].iloc[-1]
                q = returns.quantile(1 - alpha)
                value_at_risk = abs(final_value * q) * np.sqrt(timesteps)

                result = pd.DataFrame({'simulation': [simulation], 'subset': [subset], 'run': [run], 'VaR': [value_at_risk], 'q': [q]})
                results = pd.concat([results, result])

    return results.reset_index(drop=True)

In [ ]:
def calculate_VaR_threshold_probability(df, threshold):
    results = pd.DataFrame()
    
    for subset in df.subset.unique():
        df_subset = df.query("subset == @subset")
        
        df_threshold = df_subset["q"] >= threshold
        probability = df_threshold.sum() / len(df_threshold)
        
        result = pd.DataFrame({'subset': [subset], 'threshold': [threshold], 'probability': [probability]})
        results = pd.concat([results, result])
    
    return results.reset_index(drop=True)

In [ ]:
def get_data_to_plot(df, run, subset):
    pcv_ret = df.query('run == @run and subset == @subset')['total_pcv'].pct_change()
    var = df_var.query('run == @run and subset == @subset')['VaR'].iloc[0]
    q = df_var.query('run == @run and subset == @subset')['q'].iloc[0]
    
    return pcv_ret, var, q

In [ ]:
def plot_VaR_hist(df_var, variable):
    df_ = pd.concat(
        [
            df_var.query('subset==0')[variable].reset_index(drop=True),
            df_var.query('subset==1')[variable].reset_index(drop=True),
        ], axis=1)

    df_.columns = [variable+'_0', variable+'_1']
    
    return df_.hist()

In [ ]:
df_var = calculate_VaR(df2, "total_pcv", alpha=0.95, timesteps=1)

In [ ]:
df_var.query("subset == 0")[["VaR", "q"]].describe()

In [ ]:
df_var.query("subset == 1")[["VaR", "q"]].describe()

In [ ]:
plot_VaR_hist(df_var, 'VaR')

In [ ]:
plot_VaR_hist(df_var, 'q')

In [ ]:
df_var_stats_0 = df_var.query("subset == 0")[["VaR", "q"]].describe()
df_var_stats_1 = df_var.query("subset == 1")[["VaR", "q"]].describe()

print(f"1-day average PCV at Risk at 95th quantile for subset 0: \n {df_var_stats_0['VaR'].loc['mean']:,.2f} USD")
print(f"1-day average PCV at Risk at 95th quantile for subset 1: \n {df_var_stats_1['VaR'].loc['mean']:,.2f} USD")

In [ ]:
quantile_return_threshold = -0.01
q_probabilities = calculate_VaR_threshold_probability(df_var, threshold=quantile_return_threshold)

In [ ]:
for subset in q_probabilities.subset.unique():
    print(f"""For Policy {subset + 1}, the 1-Day PCV at Risk is less than {abs(quantile_return_threshold*100):.2f}% with a {100*q_probabilities.query('subset == @subset')['probability'].iloc[0]:.2f}% probability""")

In [ ]:
avg_VaR_delta = df_var_stats_0['VaR'].loc['mean'] - df_var_stats_1['VaR'].loc['mean']
avg_VaR_quantile_delta = df_var_stats_0['q'].loc['mean'] - df_var_stats_1['q'].loc['mean']

In [ ]:
print(f"The Average PCVaR Delta between parameter for policies 1 and 2 is: \n {avg_VaR_delta:,.2f} USD")
print(f"The Average PCVaR Quantile Delta between parameter for policies 1 and 2 is: \n {avg_VaR_quantile_delta:,.4f}")

In [ ]:
fig = make_subplots(rows=6, cols=2,
                    x_title='PCV Daily Returns - Left: Policy 0, Right: Policy 1',
                    y_title='Number of Observations',
                   )

for subset in [0, 1]:
    for run in range(1,7):

        pcv_ret, var, q = get_data_to_plot(df2, run, subset)

        fig.add_trace(
            px.histogram(pcv_ret, x="total_pcv", nbins=100).data[0],
            row=run, col=subset+1)

        fig.add_vline(x=q, row=run, col=subset+1)


fig.update_layout(
    title="Histogram of PCV Returns for Runs and Policy Settings",
    autosize=False,
    #width=1200,
    height=1600,
)

#fig.update_xaxes(xaxis_title='a')

fig.show()

print(f'1-Day PCVar for Run 1, Policy 0 (Subset 0) is {var:,.2f} USD with 5% quantile value {100*q:.2f}%')

# Other 

In [ ]:
# whats the probability that the state variable has a volatility greater than x for this policy setting?

In [ ]:
# whats the probability that the state variable falls below x for this policy setting?

In [ ]:
# whats the probability that the state variable ends up at at least x for this policy setting?

In [ ]:
# what is the probability that CR >= 1? VA price analysis

In [ ]:
from numpy.random import randn

In [ ]:
from scipy.stats import mannwhitneyu
from scipy.stats import wilcoxon


In [ ]:
stat, p = wilcoxon(mu1, mu2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
# interpret
alpha = 0.05
if p > alpha:
	print('Same distribution (fail to reject H0)')
else:
	print('Different distribution (reject H0)')

In [ ]:
def compute_diff_tstat(df, variable):
    
    D = dict()
    
    n_runs = len(df['run'].value_counts())
    
    for run in range(1,n_runs+1):
        diff = (df.query('subset == 0 and run==@run')[variable] -
                df2.query('subset == 1 and run==@run')[variable])
        
        mu1 = df.query('subset == 0 and run==@run')[variable].mean()
        mu2 = df.query('subset == 1 and run==@run')[variable].mean()
        
        mean_diff = mu1 - mu2
        #print('mean diff', mean_diff)

        diff_std = diff.std()
        
        t_stat = mean_diff / (diff_std / np.sqrt(n_runs))
        D[run] = t_stat
        
    return pd.DataFrame(D.values(), index=D.keys())
    

In [ ]:
compute_diff_tstat(df2, 'collateralization_ratio')#.hist(bins=30)

In [ ]:
df.query('subset == 0 and run==2')['collateralization_ratio'] - df.query('subset == 1 and run==2')['collateralization_ratio']

In [ ]:
# nonparametric hypothesis test:
# statistic - difference in - collateral ratio, stable backing ratio across pairs of policies for runs
# test: h0 Delta_v2 - Delta_v1 = 0, h1: Delta_v2 - Delta_v1 =/= 0
# https://www.investopedia.com/terms/t/t-test.asp
# https://machinelearningmastery.com/nonparametric-statistical-significance-tests-in-python/